In [15]:
%load_ext autoreload
%autoreload
import pickle
import sys
sys.path.append('..')
from utils import calc_model_importance
from preprocess import add_cluster_feature, add_feature_pca, preprocessor_trade, add_relative_distance
import pandas as pd
import numpy as np
print(np.__version__)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.19.5


In [13]:
data_dir = '../input/optiver-realized-volatility-prediction/'
file_path_trade = data_dir + "trade_train.parquet/stock_id=0"
df_feature = preprocessor_trade(file_path=file_path_trade)


   time_id  seconds_in_bucket     price  size  order_count  log_return  \
0        5                 21  1.002301   326           12         NaN   
1        5                 46  1.002778   128            4    0.000476   
2        5                 50  1.002818    55            1    0.000040   
3        5                 57  1.003155   121            5    0.000336   
4        5                 68  1.003646     4            1    0.000489   

   seconds_diff      amount  log_return_per_size  log_return_per_amount  
0           NaN  326.750235                  NaN                    NaN  
1          25.0  128.355591         3.714924e-06           3.704632e-06  
2           4.0   55.155016         7.327138e-07           7.306545e-07  
3           7.0  121.381797         2.775886e-06           2.767154e-06  
4          11.0    4.014584         1.222209e-04           1.217769e-04  


In [17]:
data = [2.0, np.nan, 3.0]
s = pd.Series(data)

5.0

In [2]:
df_train = pickle.load(open('/home/yoshikawa/work/kaggle/OPVP/output/feature_model/20210907/lgbm_add_cluster_feature/train.pkl', 'rb'))

In [5]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,...,trade_log_return_realized_volatility_100_max_time,trade_log_return_realized_volatility_100_min_time,trade_log_return_per_size_realized_volatility_100_mean_time,trade_log_return_per_size_realized_volatility_100_std_time,trade_log_return_per_size_realized_volatility_100_max_time,trade_log_return_per_size_realized_volatility_100_min_time,trade_log_return_per_amount_realized_volatility_100_mean_time,trade_log_return_per_amount_realized_volatility_100_std_time,trade_log_return_per_amount_realized_volatility_100_max_time,trade_log_return_per_amount_realized_volatility_100_min_time
0,0-5,0.004136,0.004499,7.613599e-06,0.000260,0.002292,0.006999,0.000008,0.000404,0.002325,...,0.005414,0.000932,0.000666,0.000467,0.002437,3.529954e-06,0.000665,0.000466,0.002430,3.518083e-06
1,0-11,0.001445,0.001204,1.810239e-06,0.000086,0.000360,0.002476,0.000004,0.000176,0.000801,...,0.003011,0.000000,0.000346,0.000326,0.001971,0.000000e+00,0.000346,0.000325,0.001956,0.000000e+00
2,0-16,0.002168,0.002369,-1.109201e-05,0.000173,-0.002074,0.004801,-0.000008,0.000352,-0.001493,...,0.003496,0.000770,0.000301,0.000205,0.000996,5.901960e-06,0.000301,0.000205,0.000993,5.907885e-06
3,0-31,0.002195,0.002574,-2.376661e-05,0.000236,-0.002828,0.003637,-0.000017,0.000334,-0.002053,...,0.005118,0.000285,0.000326,0.000299,0.001569,4.286066e-06,0.000326,0.000298,0.001563,4.288805e-06
4,0-62,0.001747,0.001894,-1.057099e-08,0.000144,-0.000002,0.003257,-0.000002,0.000247,-0.000281,...,0.003412,0.000346,0.000296,0.000256,0.001017,4.022623e-07,0.000296,0.000256,0.001019,4.022640e-07


In [9]:
df_train.groupby(['stock_id']).agg(np.mean)[['log_return_realized_volatility']]

,log_return_realized_volatility
stock_id,
0,0.004459
1,0.004762
2,0.002600
3,0.006876
4,0.004616
...,...
122,0.004148
123,0.002728
124,0.003989


In [8]:
stock_id_lrrv_mean = df_train.groupby(['stock_id'])['log_return_realized_volatility'].agg(np.mean)
df_train['log_return_realized_volatility_mean'] = df_train['stock_id'].map(stock_id_lrrv_mean)
df_train[['stock_id', 'log_return_realized_volatility_mean', 'log_return_realized_volatility_mean_stock']]

,stock_id,log_return_realized_volatility_mean,log_return_realized_volatility_mean_stock
0,0,0.004459,0.004459
1,0,0.004459,0.004459
2,0,0.004459,0.004459
3,0,0.004459,0.004459
4,0,0.004459,0.004459
...,...,...,...
428927,126,0.005800,0.005800
428928,126,0.005800,0.005800
428929,126,0.005800,0.005800
428930,126,0.005800,0.005800


In [10]:
mean_stock_col = [col for col in list(df_train) if 'mean_stock' in col]
mean_stock_col

['log_return_realized_volatility_mean_stock',
 'log_return2_realized_volatility_mean_stock',
 'log_return3_realized_volatility_mean_stock',
 'log_return4_realized_volatility_mean_stock',
 'log_return_realized_volatility_500_mean_stock',
 'log_return2_realized_volatility_500_mean_stock',
 'log_return3_realized_volatility_500_mean_stock',
 'log_return4_realized_volatility_500_mean_stock',
 'log_return_realized_volatility_400_mean_stock',
 'log_return2_realized_volatility_400_mean_stock',
 'log_return3_realized_volatility_400_mean_stock',
 'log_return4_realized_volatility_400_mean_stock',
 'log_return_realized_volatility_300_mean_stock',
 'log_return2_realized_volatility_300_mean_stock',
 'log_return3_realized_volatility_300_mean_stock',
 'log_return4_realized_volatility_300_mean_stock',
 'log_return_realized_volatility_200_mean_stock',
 'log_return2_realized_volatility_200_mean_stock',
 'log_return3_realized_volatility_200_mean_stock',
 'log_return4_realized_volatility_200_mean_stock',
 

In [17]:
df_test = pd.DataFrame()
df_train, df_test = add_relative_distance(df_train, df_test)
df_train

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,...,trade_log_return_per_amount_realized_volatility_400_relative,trade_log_return_realized_volatility_300_relative,trade_log_return_per_size_realized_volatility_300_relative,trade_log_return_per_amount_realized_volatility_300_relative,trade_log_return_realized_volatility_200_relative,trade_log_return_per_size_realized_volatility_200_relative,trade_log_return_per_amount_realized_volatility_200_relative,trade_log_return_realized_volatility_100_relative,trade_log_return_per_size_realized_volatility_100_relative,trade_log_return_per_amount_realized_volatility_100_relative
0,0-5,0.004136,0.004499,7.613599e-06,0.000260,0.002292,0.006999,0.000008,0.000404,0.002325,...,-0.000010,-0.000305,0.000005,0.000003,-0.000233,-0.000080,-0.000082,-0.000300,-0.000036,-0.000039
1,0-11,0.001445,0.001204,1.810239e-06,0.000086,0.000360,0.002476,0.000004,0.000176,0.000801,...,-0.000232,-0.001025,-0.000333,-0.000333,-0.001098,-0.000395,-0.000395,-0.001333,-0.000458,-0.000459
2,0-16,0.002168,0.002369,-1.109201e-05,0.000173,-0.002074,0.004801,-0.000008,0.000352,-0.001493,...,-0.000325,-0.000476,-0.000382,-0.000383,-0.000324,-0.000144,-0.000144,-0.000277,-0.000182,-0.000182
3,0-31,0.002195,0.002574,-2.376661e-05,0.000236,-0.002828,0.003637,-0.000017,0.000334,-0.002053,...,-0.000235,-0.000523,-0.000347,-0.000347,-0.000809,-0.000442,-0.000442,-0.000591,-0.000530,-0.000530
4,0-62,0.001747,0.001894,-1.057099e-08,0.000144,-0.000002,0.003257,-0.000002,0.000247,-0.000281,...,-0.000292,-0.001160,-0.000403,-0.000403,-0.001401,-0.000487,-0.000487,-0.001601,-0.000568,-0.000569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,126-32751,0.003461,0.003691,-1.706835e-06,0.000210,-0.000527,0.005876,-0.000003,0.000335,-0.000960,...,0.000075,-0.000698,0.000017,0.000018,-0.000654,-0.000096,-0.000095,-0.000879,-0.000052,-0.000051
428928,126-32753,0.003113,0.004104,1.998029e-05,0.000275,0.004436,0.004991,0.000022,0.000335,0.004829,...,0.000399,-0.000358,0.000265,0.000262,-0.000596,0.000195,0.000191,-0.000707,0.000118,0.000114
428929,126-32758,0.004070,0.003118,5.979199e-06,0.000196,0.001525,0.006019,0.000010,0.000378,0.002525,...,0.000471,-0.000569,0.000342,0.000341,-0.000842,0.000230,0.000229,-0.000935,0.000155,0.000154
428930,126-32763,0.003357,0.003661,6.429922e-07,0.000184,0.000256,0.005362,0.000002,0.000269,0.000804,...,-0.000052,-0.000629,-0.000054,-0.000056,-0.000865,-0.000124,-0.000125,-0.001025,-0.000190,-0.000192
